<a href="https://colab.research.google.com/github/dchirosca/airbnb-insights/blob/main/script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
install.packages("dplyr") # for data pre-processing
install.packages("tidyr") # for data pre-processing
install.packages("rlang") # for data pre-processing
install.packages("caret")
install.packages("randomForest")
install.packages("e1071")

library(dplyr) # for data pre-processing
library(tidyr) # for data pre-processing
library(rlang) # for data pre-processing
library(caret)
library(randomForest)
library(e1071)
options(scipen = 99)

dt<-read.csv("listings.csv")

# DATA PRE-PROCESSING -------------------------------------

# First, any column with information that will not be used in creating a model will be eliminated from the dataset

dt<-dt[, !names(dt) %in% c("listing_url",
                           "scrape_id",
                           "last_scraped",
                           "source",
                           "description",
                           "picture_url",
                           "host_url",
                           "host_name",
                           "host_location",
                           "host_thumbnail_url",
                           "host_picture_url",
                           "host_neighbourhood",
                           "host_listings_count",
                           "neighborhood_overview",
                           "neighbourhood",
                           "neighbourhood_group_cleansed",
                           "latitude",
                           "longitude",
                           "calendar_updated",
                           "license",
                           "calculated_host_listings_count",
                           "calculated_host_listins_count_entire_homes",
                           "calculated_host_listings_count_private_rooms",
                           "calculated_host_listings_count_shared_rooms",
                           "minimum_minimum_nights",
                           "maximum_minimum_nights",
                           "minimum_maximum_nights",
                           "maximum_maximum_nights",
                           "minimum_nights_avg_ntm",
                           "maximum_nights_avg_ntm",
                           "has_availability",
                           "calendar_last_scraped",
                           "number_of_reviews_ltm",
                           "number_of_review_l30d",
                           "calculated_host_listings_count_entire_homes")]

# Aiming to check the following steps to prepare the data for modelling:


# Remove duplicates -------------------------------------------------------

dt<- dt %>% distinct() # no duplicates have been identified


# Fix structural errors ---------------------------------------------------

# Correcting naming conventions
dt$host_is_superhost<-ifelse(dt$host_is_superhost == 't', 1, 0)
dt$host_has_profile_pic<-ifelse(dt$host_has_profile_pic == 't', 1, 0)
dt$host_identity_verified<-ifelse(dt$host_identity_verified == 't', 1, 0)
dt$instant_bookable<-ifelse(dt$instant_bookable == 't', 1, 0)
dt$host_response_time<-ifelse(dt$host_response_time == 'N/A', NA, dt$host_response_time)
dt$host_response_rate<-ifelse(dt$host_response_rate == 'N/A', NA, dt$host_response_rate)
dt$host_acceptance_rate<-ifelse(dt$host_acceptance_rate == 'N/A', NA, dt$host_acceptance_rate)
# Replacing a missing host description with NA for now
dt<-dt %>% mutate(host_about = replace(host_about, host_about == "", NA))
# Transforming response and acceptance rate into appropriate formats
dt$host_response_rate<-gsub("%", "", dt$host_response_rate)
dt$host_acceptance_rate<-gsub("%", "", dt$host_acceptance_rate)
dt$host_response_rate<-as.numeric(dt$host_response_rate)
dt$host_acceptance_rate<-as.numeric(dt$host_acceptance_rate)
# Transforming the price variable in numeric type
for(i in 1:dim(dt)[1]){
  dt$price[i]<-as.numeric(substr(dt$price[i], start = 2, stop = nchar(dt$price[i])))
}
# Extracting only a name for the unit, given that the name column contains more information that is otherwise captured in other columns
# Extracting the number of bedrooms, where this information is available
unit_name<-c()
for(i in 1:dim(dt)[1]){
  unit_name[i]<-strsplit(dt$name[i], split = "·")[[1]][1]
  if(grepl("bedroom", dt$name[i]) == TRUE){
    dt$bedrooms[i]<-substr(dt$name[i],
                           start = regexpr("bedroom", dt$name[i])-2,
                           stop = regexpr("bedroom", dt$name[i])-2)
  }
}
dt<-cbind(dt, unit_name)
dt<-dt[, !names(dt) == "name"]
# Extracting the number of baths
dt$bathrooms<-as.numeric(gsub("[^0-9]", "", dt$bathrooms_text)) # this will extract all numbers from text column, but without the decimal point
dt$bathrooms<-ifelse(dt$bathrooms>10, dt$bathrooms/10, dt$bathrooms) # correcting the number of baths by dividing by 10 all values greater than 10, since a property can't have more than 10 bathrooms
dt$bathrooms_text<-gsub("[0-9]", "", dt$bathrooms_text) # extracting the type of bath
# correcting the extracted type format
dt$bathrooms_text<-substr(dt$bathrooms_text, 2, nchar(dt$bathrooms_text))
dt$bathrooms_text<-trimws(dt$bathrooms_text, which = "left")
dt<-dt %>% mutate(bathrooms_text = replace(bathrooms_text, bathrooms_text == "", NA)) # replacing the missing values with NA for now
# Extracting the number of host verifications & amenities available to the guests
no_host_verifications<-c()
no_amenities<-c()
for(i in 1:dim(dt)[1]){
  no_host_verifications[i]<-length(strsplit(dt$host_verifications[i], split = ",")[[1]])
  no_amenities[i]<-length(strsplit(dt$amenities[i], split = ",")[[1]])
}
dt<-cbind(dt, no_host_verifications, no_amenities)
dt<-dt[, !names(dt) %in% c("host_verifications", "amenities")]
# Dealing with columns that contain dates and transforming them into numeric variables
class(dt$host_since) # the date is currently stored as character
dt$host_since<-as.Date(dt$host_since, format = "%Y-%m-%d")
host_seniority<-as.numeric(difftime(time1 = Sys.Date(),
                                    time2 = dt$host_since,
                                    units = "days")) # days since the host has registered as such in the airbnb system
dt<-data.frame(dt, host_seniority)
dt$first_review<-as.Date(dt$first_review, format = "%Y-%m-%d")
dt$last_review<-as.Date(dt$last_review, format = "%Y-%m-%d")
days_fr<-as.numeric(difftime(time1 = Sys.Date(),
                                    time2 = dt$first_review,
                                    units = "days")) # days since the first review
days_lr<-as.numeric(difftime(time1 = Sys.Date(),
                             time2 = dt$last_review,
                             units = "days")) # days since the last review
dt<-cbind(dt, days_fr, days_lr)
dt<-dt[, !names(dt) %in% c("host_since", "first_review", "last_review")]

# Adding additional variable to measure neighborhood safety
n_dt<-read.csv(file = "n_dt.csv")
dt<-merge(x = dt, y = n_dt, by.x = "neighbourhood_cleansed", by.y = "neighborhood", all.x = TRUE) # introduced the Human Development Index for each neighborhood to measure area safety
dt<-dt[, !names(dt) == "neighbourhood_cleansed"]

# Introducing a variable that evaluates if a property is new or existing
listing_type<-ifelse(is.na(dt$review_scores_rating) == TRUE, "new", "existing")
dt<-cbind(dt, listing_type)

# Introducing new features related to the host description
description_no_words<-c()
for(i in 1:dim(dt)[1]){
  if(is.na(dt$host_about[i]) == TRUE){
    description_no_words[i]<-0
  } else{
    description_no_words[i]<-length(strsplit(dt$host_about[i], " ")[[1]])
  }
}
dt<-cbind(dt, description_no_words)

# Transforming availability in a more suitable format
dt$availability_30<-round(dt$availability_30/30,4)
dt$availability_60<-round(dt$availability_60/60,4)
dt$availability_90<-round(dt$availability_90/90,4)
dt$availability_365<-round(dt$availability_365/365,4)

# Dropping any last set of columns that cannot be used in the analysis
dt<-dt[, !names(dt) %in% c("id",
                           "host_id",
                           "host_about",
                           "number_of_reviews_l30d",
                           "review_scores_accuracy",
                           "review_scores_cleanliness",
                           "review_scores_checkin",
                           "review_scores_communication",
                           "review_scores_location",
                           "review_scores_value",
                           "unit_name",
                           "property_type")]

dt<-dt %>% mutate_at(vars(host_response_rate,
                 host_acceptance_rate,
                 host_total_listings_count,
                 accommodates,
                 bathrooms,
                 bedrooms,
                 beds,
                 price,
                 minimum_nights,
                 maximum_nights,
                 availability_30,
                 availability_60,
                 availability_90,
                 availability_365,
                 number_of_reviews,
                 review_scores_rating,
                 reviews_per_month,
                 no_host_verifications,
                 no_amenities,
                 host_seniority,
                 days_fr,
                 days_lr,
                 HI_index,
                 description_no_words), as.numeric)

# Dealing with missing values ---------------------------------------------

# Rows that do not have any value for the variables that describe the property, will be eliminated
print(paste0("The initial number of rows is ", dim(dt)[1])) # 39627
# Separating the new properties from the existing ones
df<-dt %>% filter(listing_type == "new")
dt<-dt %>% filter(listing_type == "existing") # keeping only the existing properties who had at least one guest
dt<-dt %>% drop_na()
dt<-dt[, !names(dt) == "listing_type"]

# Data encoding -----------------------------------------------------------

# Transforming the character variables in categorial variables
dt$host_response_time<-as.factor(dt$host_response_time)
dt$room_type<-as.factor(dt$room_type)
dt$bathrooms_text<-as.factor(dt$bathrooms_text)


# **Random forest with grid search**

The principle of grid search is quite simple: the model will be evaluated over all the combinations passed to the function, using cross-validation.
For this analysis, we will implement random search - this method will not evaluate all the combinations of hyperparameters in the searching space all at once, but it will randomly choose combinations at every iteration. This method has the great advantage of lower computational cost compared to the original grid search method.
We will follow these steps:
*   Find the best number of mtry
*   Find the best number of maxnodes
*   Find the best number of ntrees
*   Evaluate the optimal model

Afterwards, we will also vizualize the importance of the variables.
The best number of mtry, maxnodes and ntrees will be chosen by minimizing the RMSE generated by the model.
For the evaluation of the model, we will use a **K-fold cross validation**, which will be implemented using the trainControl() function from R. For this function, we will use:
*   **method** = "cv"; this is the method used to resample the dataset and is referring to cross-validation. In cross-validation, the dataset is randomly divided into k subsets (also called folds). For each iteration, one subset is kept as the validation data and the remaining k-1 are used as training data. K results will be obtained and they will be averaged to produce one single estimation.
*   **number** = n; the number of folds to create. We will choose a default number of 10 folds that will offer a good balance between computational efficiency and reliable estimates created by the model
*   **search** = "grid" to use the search grid method

The randomForest algorithm has the following structure in R:
*RandomForest(formula, ntree, mtry, maxnodes)*, where:
*   **formula**: the formula of the fitted model which will be review_scores_rating~. to use review_scores_rating as a function of all the features included in the dataset
*   **ntree**: the number of trees in the forest; generally, the more trees, the better the model performance, but the improvements will plateau after a certain point. Usually, a number between 100 and 1000 is used to train random forest models
*   **mtry**: the number of variable candidates draw to feed the algorithm
*   **maxnodes**: the maximum amount of terminal nodes in the forest; large numbers of terminal trees tend to increase the model complexity, but also might lead to overfitting










In [ ]:
# Random forest -----------------------------------------------------------

# Searching for the best mtry with grid search
set.seed(1234)
tuneGrid<-expand.grid(.mtry = c(10: 20)) # starting with one-third of the total number of variables
rf_default<-train(review_scores_rating~.,
                  data = dt,
                  method = "rf",
                  metric = "rmse",
                  trControl = trainControl(method = "cv",
                                             number = 10,
                                             search = "grid"),
                  tuneGrid = tuneGrid,
                  importance = TRUE)
print(rf_default)

rf_default$bestTune$mtry # mtry = 11
min(rf_default$results$RMSE) # the lowest RMSE = 0.206375387857852

best_mtry<-rf_default$bestTune$mtry
best_mtry

# Searching for the best maxnodes
store_maxnode <- list()
tuneGrid <- expand.grid(.mtry = best_mtry)
for (maxnodes in c(5:30)) {
    set.seed(1234)
    rf_maxnode <- train(review_scores_rating~.,
        data = dt,
        method = "rf",
        metric = "RMSE",
        tuneGrid = tuneGrid,
        trControl = trainControl(method = "cv",
                                             number = 10,
                                             search = "grid"),
        importance = TRUE,
        nodesize = 14,
        maxnodes = maxnodes,
        ntree = 300)
    key<-toString(maxnodes)
    store_maxnode[[key]] <- rf_maxnode
}
results_node <- resamples(store_maxnode)
summary(results_node) # maxnodes = 29 has the lowest RMSE value of 0.2106723, so we will continue with it

# Searching the best ntrees
store_maxtrees <- list()
tuneGrid <- expand.grid(.mtry = best_mtry)
for (ntree in c(250, 300, 350, 400, 450, 500, 550, 600, 800, 1000, 2000)) {
    set.seed(1234)
    rf_maxtrees <- train(review_scores_rating~.,
        data = dt,
        method = "rf",
        metric = "RMSE",
        tuneGrid = tuneGrid,
        trControl = trainControl(method = "cv",
                                             number = 10,
                                             search = "grid"),
        importance = TRUE,
        nodesize = 14,
        maxnodes = 29,
        ntree = ntree)
    key <- toString(ntree)
    store_maxtrees[[key]] <- rf_maxtrees
}
results_tree <- resamples(store_maxtrees)
summary(results_tree) # will train a model with ntrees = 400 which returns a RMSE of 0.2106102

# Creating the first model
set.seed(1234)
tuneGrid<-expand.grid(.mtry = best_mtry)
fit_rf<-randomForest(review_scores_rating~.,
    data = dt,
    method = "rf",
    metric = "RMSE",
    tuneGrid = tuneGrid,
    trControl = trainControl(method = "cv",
                                             number = 10,
                                             search = "grid"),
    importance = TRUE,
    nodesize = 14,
    ntree = 400,
    maxnodes = 29)
summary(fit_rf)
sqrt(mean(fit_rf$mse))

install.packages("ggRandomForests")
library(ggRandomForests)
error_plot<-gg_error(fit_rf)
plot(error_plot)

# Vizualizing variable importance
varImpPlot(fit_rf)


ERROR: Error in gg_cle(fit_rf, pred.var = "HI_index"): could not find function "gg_cle"


# **XGBoost**

To get the dataset ready to be fed into the xgboost function, we needed to:

Remove information about the target variable from the

*   Remove information about the target variable from the training data
*   Reduce the redudant information (*done in the first part of the script*)
*   Convert categorial information to a numeric format
*   Convert the dataframes into Dmatrixes

In [ ]:
# xgboost -----------------------------------------------------------
install.packages("xgboost")
library(xgboost)
library(plyr)

# Preparing the data for xgboost implementation
# Convert categorical information to a numeric format
dt$host_response_time<-as.character(dt$host_response_time)
dt$host_response_time<-mapvalues(dt$host_response_time,
                              from = unique(dt$host_response_time),
                              to = c(1:length(unique(dt$host_response_time))))
dt$host_response_time<-as.numeric(dt$host_response_time)
dt$room_type<-as.character(dt$room_type)
dt$room_type<-mapvalues(dt$room_type,
                              from = unique(dt$room_type),
                              to = c(1:length(unique(dt$room_type))))
dt$room_type<-as.numeric(dt$room_type)
dt$bathrooms_text<-as.character(dt$bathrooms_text)
dt$bathrooms_text<-mapvalues(dt$bathrooms_text,
                              from = unique(dt$bathrooms_text),
                              to = c(1:length(unique(dt$bathrooms_text))))
dt$bathrooms_text<-as.numeric(dt$bathrooms_text)

# Spliting dataset into trainging and testing subsets
set.seed(1234)
splitIndex<-createDataPartition(dt$review_scores_rating, p = 0.75, list = FALSE) # splitting the dataset in 75-25 training and test
train_data<-dt[splitIndex,]
test_data<-dt[-splitIndex,]

# Separate features and labels
train_labesl<-train_data$review_scores_rating
train_data<-train_data[, -which(names(train_data) == "review_scores_rating")]

test_labels<-test_data$review_scores_rating
test_data<-test_data[, -which(names(test_data) == "review_scores_rating")]

# Convert the cleaned dataframe to a Dmatrix
dtrain<-xgb.DMatrix(data = as.matrix(train_data), label = train_labels)
dtest<-xgb.DMatrix(data = as.matrix(test_data), label = test_labels)

# **Training the XGBoost Model**

We will first train one model and then work on optimizing the parameters.
Since our business problem is of regression type, there are multiple options to choose from:
*   reg:squarederror (Mean Squared Error) - this is used when the dataset has a Gaussian error distribution and is the most common choice for regression problems
*   reg:squaredlogerror (Mean Squared Logarithmic Error) - this is used when the target variable is positive and it is desired to penalize underestimates more than overestimates; this type of regression is particularly useful when dealing with growth rates
*   reg:absoluteerror (Mean Absolute Error) - this is especially used when the data has outliers or non-normal error distribution; this type of regression will treat all the errors the same regardless of their size
*   reg:quantileerror (Quantile Loss) - this type is used when it is desired that a certain percentile of the distribution of the target variable will also be predicted, aside from the mean

Due to the presence of outliers and the target variable being only positive, we will choose squaredlogerror

We will start with a learning rate (eta) of 0.3, a maximum depth of a tree of 6



In [ ]:
params<-list(
  objective = "reg:squaredlogerror",
  eta = 0.3,
  max_depth = 6
)

set.seed(1234)  # for reproducibility
xgb_model<-xgb.train(params, dtrain, nrounds = 1000)
preds<-predict(xgb_model, dtest)
preds
rmse<-sqrt(mean((preds - test_labels)^2))
print(rmse) # we obtain a RMSE of 0.2359304, which currently is higher than the value from the randomForest model
plot(test_labels, preds, xlim = c(4,5), ylim = c(4,5), xlab = "observed", ylab = "predicted")

# Further, we will hyperparameter tune the xgboost model

# creating a grid of hyperparameters
nrounds<-seq(from = 100, to = 1000, by = 100)
eta<-c(0.05, 0.1, 0.3, 0.5)
max_depth<-c(6, 8, 10)
gamma<-c(0, 0.1, 0.2)

rmse_values<-c()
model_params<-c()
for(i in 1:length(nrounds)){
  for(j in 1:length(eta)){
    for(m in 1:length(max_depth)){
      for(n in 1:length(gamma)){
        params<-list(objective = "reg:squaredlogerror",
                      eta = eta[j],
                      max_depth = max_depth[m],
                      gamma = gamma[n]
                    )
        set.seed(1234)  # for reproducibility
        xgb_model<-xgb.train(params, dtrain, nrounds = nrounds[i])
        preds<-predict(xgb_model, dtest)
        rmse<-sqrt(mean((preds - test_labels)^2))
        rmse_values<-c(rmse_values, rmse)
        model_params<-c(model_params, paste0("nrounds", nrounds[i], "eta", eta[j], "max_depth", max_depth[m], "gamma", gamma[n]))
      }
    }
  }
}
xgb_models<-data.frame(model_params, rmse_values)
min(xgb_models$rmse_values) # RMSE = 0.2180775
xgb_models[which.min(xgb_models$rmse_values),] # nrounds = 200, eta = 0.05, max_depth = 8, gamma = 0

params<-list(
  objective = "reg:squaredlogerror",
  eta = 0.05,
  max_depth = 8
)
set.seed(1234)  # for reproducibility
opt_xgb_model<-xgb.train(params, dtrain, nrounds = 200)
preds<-predict(opt_xgb_model, dtest)
rmse<-sqrt(mean((preds - test_labels)^2))
print(rmse) # we obtain a RMSE of 0.2180775, which currently is higher than the value from the randomForest model
plot(test_labels, preds, xlim = c(4,5), ylim = c(4,5), xlab = "observed", ylab = "predicted")

# Vizualizing variable importance
importance_matrix<-xgb.importance(feature_names = colnames(train_data), model = opt_xgb_model)
xgb.plot.importance(importance_matrix)

[1] 360